In [45]:
import os
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt



In [46]:
# Load the background image
background= cv2.imread("back.jpg")
background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
background= cv2.cvtColor(background, cv2.COLOR_RGB2HSV)
BH, BS, BV= cv2.split(background)
BV= np.float32(BV)/255
BS= np.float32(BS)/255
BH= np.float32(BH)/255

In [47]:
alpha1 = 0.2 ; beta1 = 0.8 ; thetas= 1; thetah= 1
kernel = np.ones((3,3), np.uint8)

In [ ]:
path1 = "/home/user/Downloads/DAY2/SET7/"    
path2 = "/home/user/Downloads/DAY2/SET7OUT/"   
listing = os.listdir(path1)    
for file in listing:
    img = cv2.imread(path1+file)
    outfile= path2+file
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image=img
    
    #plt.subplot(231)
    #plt.imshow(image, cmap = 'gray')
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    H, S, V = cv2.split(img)
    V = np.float32(V)/255
    S = np.float32(S)/255
    H = np.float32(H)/255
    
    s= V.copy()
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            try:
                x= V[i,j]/ BV[i,j]
            except ZeroDivisionError:
                x = 0
        
        if ( (alpha1 <= x <= beta1) and (abs(S[i,j]-BS[i,j])<= thetas) and (abs(H[i,j]-BH[i,j])<=thetah)) :
            s[i,j]=1
        else:
            s[i,j]=0
            
    s=s*255
   
    s = np.array(s,dtype=np.uint8)
    #plt.subplot(232)
    plt.imshow(s, cmap = 'gray')
    median = cv2.medianBlur(s, 5)
    closing = cv2.morphologyEx(median, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    opening = np.array(opening,dtype=np.uint8)
    #plt.subplot(233)
    plt.imshow(opening, cmap = 'gray')
   
    result = image.copy()
    height=result.shape[0]
    width=result.shape[1]
    blank_image = np.zeros((height,width,3), np.uint8)
    blank_image[opening!=0] = image[opening!=0]
    #plt.subplot(234)
    plt.imshow(blank_image, cmap = 'gray')
    blank_image1=np.uint8(np.double(blank_image)*1.2 + 5)
    #plt.subplot(235)
    plt.imshow(blank_image1, cmap = 'gray')
    claheimg = np.zeros((height,width,3), np.uint8)
    claheimg[opening==0]=image[opening==0]
    clahe = cv2.createCLAHE(clipLimit=9.0,tileGridSize=(5, 5))

    claheimg[:,:,0] = clahe.apply(blank_image1[:,:,0])
    claheimg[:,:,1] = clahe.apply(blank_image1[:,:,1])
    claheimg[:,:,2] = clahe.apply(blank_image1[:,:,2])
    cv2.imwrite(outfile,claheimg)

KeyboardInterrupt: 